In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, make_scorer
import lightgbm as lgb
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd

In [8]:
df = pd.read_csv('../../dataset/final/merged_dataset_final_all_pp.csv')

In [9]:
df.columns

Index(['latitude', 'longitude', 'nearest_lat', 'nearest_lon', 'distance',
       'year', 'month', 'day', 'tp_mm', 't2m_C', 'anomaly_T2m_C',
       'heat_stress_proxy', 'tp_7d_cum', 'tp_14d_cum', 'tp_7d_avg',
       'consec_rain_days', 'tp_lag1', 'tp_lag3', 'tp_lag7', 'heavy_rain',
       'month_sin', 'month_cos', 'heat_proxy', 'heat_next_day',
       'heat_next_2days', 'heat_next_3days', 'tp_anomaly', 'tp_std_anomaly',
       'heatwave_flag', 'next_day_match', 'next_2days_match',
       'next_3days_match', 'lat', 'lon', 'ph', 'organic_matter',
       'total_nitrogen', 'potassium', 'p2o5', 'boron', 'zinc', 'sand', 'clay',
       'slit', 'parentsoil', 'province', 'district', 'palika', 'crop',
       'variety', 'UREA1', 'UREA2', 'UREA3', 'DAP', 'MOP', 'organic',
       'boron_fert', 'palika_num'],
      dtype='object')

In [30]:
df.drop(columns='palika_num', inplace=True)
df.drop(columns='district', inplace=True)
df.drop(columns='province', inplace=True)
df.drop(columns='palika', inplace=True)




In [31]:
corr = y_eval.corr()
corr

,UREA1,UREA2,UREA3,DAP,MOP
UREA1,1.000000,0.713155,0.713155,-0.341633,0.128046
UREA2,0.713155,1.000000,1.000000,0.415166,0.614745
UREA3,0.713155,1.000000,1.000000,0.415166,0.614745
DAP,-0.341633,0.415166,0.415166,1.000000,0.656673
MOP,0.128046,0.614745,0.614745,0.656673,1.000000


In [32]:
features = [
    'latitude', 'longitude', 'distance',
    'year', 'month', 'day', 'tp_mm', 't2m_C', 'ph', 'organic_matter', 
    'total_nitrogen', 'potassium', 'p2o5', 'boron', 'zinc', 'sand', 
    'clay', 'slit', 'parentsoil', 'variety', 'crop'
]
targets = ['UREA1','UREA2','UREA3','DAP','MOP']

In [33]:

X = df[features].copy()
y = df[targets].copy()

numeric_cols = X.select_dtypes(include=['float64','int64']).columns

# -------- Models to compare --------
models_to_compare = {
    'RandomForest': RandomForestRegressor(n_estimators=300, max_depth=15, random_state=42),
    'ExtraTrees': ExtraTreesRegressor(n_estimators=300, max_depth=15, random_state=42),
    'LightGBM': lgb.LGBMRegressor(n_estimators=300, max_depth=15, learning_rate=0.05, random_state=42),
    'XGBoost': xgb.XGBRegressor(n_estimators=300, max_depth=7, learning_rate=0.05, random_state=42)
}


In [34]:
print(X.columns)
print(y.columns)

Index(['latitude', 'longitude', 'distance', 'year', 'month', 'day', 'tp_mm',
       't2m_C', 'ph', 'organic_matter', 'total_nitrogen', 'potassium', 'p2o5',
       'boron', 'zinc', 'sand', 'clay', 'slit', 'parentsoil', 'variety',
       'crop'],
      dtype='object')
Index(['UREA1', 'UREA2', 'UREA3', 'DAP', 'MOP'], dtype='object')


In [35]:
# Example: check correlation of X features with targets
import seaborn as sns
import matplotlib.pyplot as plt

corr = X.corrwith(y_eval['UREA1'])
corr

c:\Users\Acer\.conda\envs\nlp-env\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\Acer\.conda\envs\nlp-env\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


latitude         -0.268675
longitude        -0.141659
distance         -0.076969
year             -0.088070
month            -0.025720
day              -0.000930
tp_mm             0.027660
t2m_C             0.169270
ph                0.037573
organic_matter   -0.201816
total_nitrogen   -0.430731
potassium         0.401062
p2o5              0.621016
boron             0.189517
zinc             -0.010180
sand             -0.032143
clay             -0.553906
slit              0.296526
parentsoil       -0.382320
variety          -0.241732
crop                   NaN
dtype: float64

In [36]:
y.describe()


,UREA1,UREA2,UREA3,DAP,MOP
count,26475.000000,26475.000000,26475.000000,26475.000000,26475.000000
mean,1.367391,2.046811,2.046811,1.751775,1.686598
std,0.361204,0.376305,0.376305,0.717827,0.420499
min,0.970000,1.810000,1.810000,0.540000,1.250000
25%,1.180000,1.810000,1.810000,1.630000,1.340000
50%,1.180000,1.810000,1.810000,1.630000,1.670000
75%,1.720000,2.420000,2.420000,2.170000,1.670000
max,2.210000,3.620000,3.620000,3.620000,3.340000


In [37]:
df.corr()['UREA1'].sort_values(ascending=False)


UREA1                1.000000
UREA3                0.713155
UREA2                0.713155
p2o5                 0.621016
potassium            0.401062
slit                 0.296526
boron                0.189517
anomaly_T2m_C        0.169270
t2m_C                0.169270
MOP                  0.128046
heat_stress_proxy    0.119000
heatwave_flag        0.116199
lon                  0.085422
nearest_lon          0.085422
tp_14d_cum           0.059904
tp_7d_avg            0.050390
tp_7d_cum            0.050390
ph                   0.037573
consec_rain_days     0.030046
tp_mm                0.027660
tp_lag1              0.027579
tp_lag3              0.027487
tp_lag7              0.027156
tp_anomaly           0.016737
tp_std_anomaly       0.002845
heavy_rain           0.002369
day                 -0.000930
month_sin           -0.002622
heat_next_3days     -0.003025
heat_next_2days     -0.003529
heat_next_day       -0.004050
heat_proxy          -0.004205
zinc                -0.010180
month     

In [38]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X_train, X_test, y_train, y_test = train_test_split(X, y['UREA1'], test_size=0.2, random_state=42)
model = LinearRegression().fit(X_train, y_train)
r2_score(y_test, model.predict(X_test))


0.6421388246839002

In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor

# -------- Features & Targets --------
X = df[['latitude', 'longitude', 'distance', 'year', 'month', 'day', 'tp_mm',
        't2m_C', 'ph', 'organic_matter', 'total_nitrogen', 'potassium', 'p2o5',
        'boron', 'zinc', 'sand', 'clay', 'slit', 'parentsoil', 'variety', 'crop']]

y = df[['UREA1', 'UREA2', 'DAP', 'MOP']]  # dropped UREA3 (duplicate)

# -------- Encode categorical features --------
X = pd.get_dummies(X, columns=['parentsoil', 'variety', 'crop'], drop_first=True)

# -------- GroupKFold by Crop --------
groups = df['crop']
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# -------- Model Setup --------
base_model = LGBMRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.3,
    reg_lambda=0.7,
    random_state=42
)

model = MultiOutputRegressor(base_model)

# -------- Cross-validation --------
rmse_scores, mae_scores, r2_scores = [], [], []

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y, groups)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    rmse_scores.append(rmse)
    mae_scores.append(mae)
    r2_scores.append(r2)

    print(f"Fold {fold+1}: RMSE={rmse:.4f}, MAE={mae:.4f}, R²={r2:.4f}")

print("\n=== Mean Performance ===")
print(f"RMSE={np.mean(rmse_scores):.4f}")
print(f"MAE={np.mean(mae_scores):.4f}")
print(f"R²={np.mean(r2_scores):.4f}")


c:\Users\Acer\.conda\envs\nlp-env\Lib\site-packages\sklearn\model_selection\_split.py:86: UserWarning: The groups parameter is ignored by KFold
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001396 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1106
[LightGBM] [Info] Number of data points in the train set: 21180, number of used features: 22
[LightGBM] [Info] Start training from score 1.367010
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

In [ ]:
# Drop constant columns in y for evaluation
constant_cols = y.columns[y.std() == 0]
prasdint("Dropping constant targets:", list(constant_cols))
y_eval = y.drop(columns=constant_cols)

# Cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)
results = []

for model_name, base_model in models_to_compare.items():
    print(f"\nEvaluating {model_name} ...")
    fold_rmse = []
    fold_mae = []
    fold_r2 = []

    # Wrap models that cannot handle multi-output natively
    if model_name in ['XGBoost', 'LightGBM']:
        model = MultiOutputRegressor(base_model)
    else:
        model = base_model

    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X.iloc[train_idx].copy(), X.iloc[test_idx].copy()
        y_train, y_test = y_eval.iloc[train_idx], y_eval.iloc[test_idx]

        # Scale numeric features
        scaler = StandardScaler()
        X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
        X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

        # Fit model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Compute per-target metrics
        rmse_per_target = np.sqrt(np.mean((y_test.values - y_pred)**2, axis=0))
        mae_per_target = np.mean(np.abs(y_test.values - y_pred), axis=0)
        r2_per_target = r2_score(y_test.values, y_pred, multioutput='raw_values')

        fold_rmse.append(rmse_per_target)
        fold_mae.append(mae_per_target)
        fold_r2.append(r2_per_target)

    # Average metrics across folds
    mean_rmse_per_target = np.mean(fold_rmse, axis=0)
    mean_mae_per_target = np.mean(fold_mae, axis=0)
    mean_r2_per_target = np.mean(fold_r2, axis=0)

    results.append({
        'Model': model_name,
        'Mean RMSE': np.mean(mean_rmse_per_target),
        'Mean MAE': np.mean(mean_mae_per_target),
        'Mean R2': np.mean(mean_r2_per_target)
    })

    print(f"{model_name:20s} | RMSE per target: {mean_rmse_per_target}")
    print(f"{model_name:20s} | MAE  per target: {mean_mae_per_target}")
    print(f"{model_name:20s} | R2   per target: {mean_r2_per_target}")
    print(f"{model_name:20s} | Mean RMSE: {np.mean(mean_rmse_per_target):.4f}, Mean MAE: {np.mean(mean_mae_per_target):.4f}, Mean R2: {np.mean(mean_r2_per_target):.4f}\n")

# Convert results to DataFrame for comparison
results_df = pd.DataFrame(results).sort_values('Mean RMSE')
print("\n=== Model Comparison ===")
print(results_df)


Dropping constant targets: ['boron_fert']

Evaluating RandomForest ...
RandomForest         | RMSE per target: [5.11104434e-04 5.11104434e-04 5.11104434e-04 4.21449750e-14
 1.34965812e-05]
RandomForest         | MAE  per target: [1.20302172e-05 1.20302172e-05 1.20302172e-05 3.01817117e-14
 2.90840442e-07]
RandomForest         | R2   per target: [0.99999056 0.99999091 0.99999091 1.         1.        ]
RandomForest         | Mean RMSE: 0.0003, Mean MAE: 0.0000, Mean R2: 1.0000


Evaluating ExtraTrees ...
ExtraTrees           | RMSE per target: [2.21339263e-05 1.55511483e-05 1.55511483e-05 3.46312015e-05
 2.24443473e-14]
ExtraTrees           | MAE  per target: [5.55240806e-07 3.70160543e-07 3.70160543e-07 4.75920703e-07
 1.74662089e-14]
ExtraTrees           | R2   per target: [0.99999999 0.99999999 0.99999999 0.99999999 1.        ]
ExtraTrees           | Mean RMSE: 0.0000, Mean MAE: 0.0000, Mean R2: 1.0000


Evaluating LightGBM ...
[LightGBM] [Info] Auto-choosing row-wise multi-threading,

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("R² =", r2_score(y_test, y_pred))
print("RMSE =", np.sqrt(mean_squared_error(y_test, y_pred)))
print("MAE =", mean_absolute_error(y_test, y_pred))


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001217 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1106
[LightGBM] [Info] Number of data points in the train set: 21180, number of used features: 22
[LightGBM] [Info] Start training from score 1.367010
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

In [50]:
corr = X.join(y)['total_nitrogen'].corr(y['MOP'])
print("UREA1–UREA3 correlation:", corr)


UREA1–UREA3 correlation: 0.32705828290916417


In [51]:
corrs = X.corrwith(y['UREA1']).sort_values(ascending=False)
print(corrs.head(10))


p2o5              0.621016
potassium         0.401062
slit              0.296526
parentsoil_1.0    0.246319
boron             0.189517
t2m_C             0.169270
ph                0.037573
tp_mm             0.027660
day              -0.000930
zinc             -0.010180
dtype: float64


In [17]:
df.dtypes


latitude             float64
longitude            float64
nearest_lat          float64
nearest_lon          float64
distance             float64
year                   int64
month                  int64
day                    int64
tp_mm                float64
t2m_C                float64
anomaly_T2m_C        float64
heat_stress_proxy      int64
tp_7d_cum            float64
tp_14d_cum           float64
tp_7d_avg            float64
consec_rain_days       int64
tp_lag1              float64
tp_lag3              float64
tp_lag7              float64
heavy_rain             int64
month_sin            float64
month_cos            float64
heat_proxy             int64
heat_next_day          int64
heat_next_2days        int64
heat_next_3days        int64
tp_anomaly           float64
tp_std_anomaly       float64
heatwave_flag          int64
next_day_match          bool
next_2days_match        bool
next_3days_match        bool
lat                  float64
lon                  float64
ph            

In [15]:
print(y.describe())


              UREA1         UREA2         UREA3           DAP           MOP  \
count  26475.000000  26475.000000  26475.000000  26475.000000  26475.000000   
mean       1.367391      2.046811      2.046811      1.751775      1.686598   
std        0.361204      0.376305      0.376305      0.717827      0.420499   
min        0.970000      1.810000      1.810000      0.540000      1.250000   
25%        1.180000      1.810000      1.810000      1.630000      1.340000   
50%        1.180000      1.810000      1.810000      1.630000      1.670000   
75%        1.720000      2.420000      2.420000      2.170000      1.670000   
max        2.210000      3.620000      3.620000      3.620000      3.340000   

       boron_fert  
count     26475.0  
mean          1.0  
std           0.0  
min           1.0  
25%           1.0  
50%           1.0  
75%           1.0  
max           1.0  
